# Examples of usage.


## 1. Extracting a complete synthetic route from a research document (paper+SI).

In [ ]:
import os
import pickle

import dspy
from dotenv import load_dotenv

from jasyntho import SynthTree
from jasyntho.utils import Mistral

load_dotenv()
mistral_key = os.getenv("MISTRAL_API_KEY")

# Load a language model
language_model = Mistral(
    model="mistral-small-latest",
    api_key=mistral_key,
    api_base="https://api.mistral.ai/v1/",
    temperature=0.1,
    max_tokens=1000,
)
dspy.settings.configure(lm=language_model)

In [ ]:
# 1. Create new tree object

paper_src = "../data/1c10539"
tree = SynthTree.from_dir(paper_src)

In [ ]:
# 2. Extract products list from original tree object (extract_rss)

# tree.raw_prods = await tree.extract_rss()
# tree.products = [p for p in tree.raw_prods if not p.isempty()]

with open(os.path.join(paper_src, "raw_prods_full.tree"), "rb") as f:
    tree.raw_prods = pickle.load(f)
    tree.products = [p for p in tree.raw_prods if not p.isempty()]

In [ ]:
# 3. # OPTIONAL # . Get list of reachable subgraphs (this gives you a list of connected graphs!) (tree.disjoint_trees())
disjoint_trees = tree.get_reachable_subgraphs()

# 4. For each disjoint tree, find extra experimental connections. (make function for this)
new_connects = tree.extended_connections()

# Here we're basically trying to build more edges between substances that couldn't be captured with the basic connection extraction (step 2)
# This new_connects object returns the reasoning behind the new connections
print(new_connects)

In [ ]:
# 4.1 You can always visualize the extracted trees accessing tree.list_disjoint

dt = tree.list_disjoint

import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout


def plot_graph(G):
    fig = plt.figure(figsize=(10, 7))
    pos = graphviz_layout(G, prog="dot")
    nx.draw(G, pos, with_labels=True, arrows=True)


for g in dt.values():
    if len(g) > 1:
        plot_graph(g)

In [ ]:
# 5. Get smiles for nodes in tree (only product nodes)
tree.gather_smiles()

In [ ]:
# 6. Export into usable json format
# This format is adapted to aizynthfinder's synthesis format. Useful for visualization, GED calcs, etc.

json_format = tree.export()  # gets a json for each disjoint tree

In [ ]:
# 7. Visualize
from rxn_tree_vis.reaction.reactiontree import ReactionTree

key = "28"
t = ReactionTree.from_dict(json[key])
im = t.to_image()
im.save(f"img_{key}.png")